<a href="https://colab.research.google.com/github/fraziermatthew/ImageAI/blob/master/imageaimodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/usr/local/src')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /usr/local/src


In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**Step 1**

Install all dependencies: 

* tensorflow
* numpy
* scipy
* opencv-python
* pillow
* matplotlib
* h5py
* keras
* ImageAI
* the RetinaNet model file that will be used for object detection to /usr/local/src/



In [0]:
!pip3 install tensorflow
!pip3 install numpy
!pip3 install scipy
!pip3 install opencv-python
!pip3 install pillow
!pip3 install matplotlib
!pip3 install h5py
!pip3 install keras
!pip3 install https://github.com/OlafenwaMoses/ImageAI/releases/download/2.0.2/imageai-2.0.2-py3-none-any.whl

In [0]:
mkdir -p /usr/local/src/pets/train/cats

In [0]:
mkdir -p /usr/local/src/pets/train/dogs

In [0]:
mkdir -p /usr/local/src/pets/test/cats

In [0]:
mkdir -p /usr/local/src/pets/test/dogs

In [0]:
cd /usr/local/src/

/usr/local/src


Train the model

In [0]:
from imageai.Prediction.Custom import ModelTraining

model_trainer = ModelTraining()
model_trainer.setModelTypeAsResNet()
model_trainer.setDataDirectory("/usr/local/src/pets")
model_trainer.trainModel(num_objects=2, num_experiments=100, enhance_data=True, batch_size=32, show_network_summary=True)


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 112, 112, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 112, 112, 64) 0           batch_normalization_v1[0][0]     
_____________________________________

#ResNet Prediction
Simple image prediction - Rotated Image 45 degrees

In [0]:
from imageai.Prediction import ImagePrediction
import os

execution_path = os.getcwd()

prediction = ImagePrediction()
prediction.setModelTypeAsResNet()
prediction.setModelPath(os.path.join(execution_path, "resnet50_weights_tf_dim_ordering_tf_kernels.h5"))
prediction.loadModel()

predictions, probabilities = prediction.predictImage(os.path.join(execution_path, "pets/test/cats/cat5.jpg"), result_count=10)
for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)

tabby  :  35.23553013801575
fur_coat  :  24.137982726097107
marmoset  :  7.62396827340126
Windsor_tie  :  6.486064940690994
Egyptian_cat  :  3.3730898052453995
solar_dish  :  2.1897170692682266
tiger_cat  :  1.4715631492435932
envelope  :  1.2845500372350216
stole  :  1.115142647176981
muzzle  :  0.9159897454082966


# Custom Prediction

Single image prediction - Rotated Image 45 degrees

In [0]:
from imageai.Prediction.Custom import CustomImagePrediction
import os

execution_path = os.getcwd()

prediction = CustomImagePrediction()
prediction.setModelTypeAsResNet()
prediction.setModelPath(os.path.join(execution_path, "models/model_ex-002_acc-0.495000.h5"))
prediction.setJsonPath(os.path.join(execution_path, "json/model_class.json"))
prediction.loadModel(num_objects=2)

predictions, probabilities = prediction.predictImage(os.path.join(execution_path, "test/cats/cat5.jpg"), result_count=5)

for eachPrediction, eachProbability in zip(predictions, probabilities):
    print(eachPrediction , " : " , eachProbability)

cats  :  77.10055708885193
dogs  :  22.899439930915833


Multiple image Prediction

In [0]:
from imageai.Prediction.Custom import CustomImagePrediction
import os

execution_path = os.getcwd()

multiple_prediction = CustomImagePrediction()
multiple_prediction.setModelTypeAsResNet()
multiple_prediction.setModelPath(os.path.join(execution_path, "models/model_ex-005_acc-0.495000.h5"))
multiple_prediction.setJsonPath(os.path.join(execution_path, "json/model_class.json"))
multiple_prediction.loadModel(num_objects=2)

all_images_array = []
path = "/usr/local/src/pets/test/dogs/"
all_files = os.listdir(path)

for each_file in all_files:
    if(each_file.endswith(".jpg")):
        all_images_array.append(each_file)

results_array = multiple_prediction.predictMultipleImages(all_images_array, result_count_per_image=2)

for each_result in results_array:
    predictions, percentage_probabilities = each_result["predictions"], each_result["percentage_probabilities"]
    for index in range(len(predictions)):
        print(predictions[index] , " : " , percentage_probabilities[index])
    print("-----------------------")

In [0]:
cd /usr/local/src/pets

/usr/local/src/pets


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
import cv2
import os, os.path


path = "/usr/local/src/pets/train/cats/cat"
path2 = "/usr/local/src/pets/train/dogs/dog"
i = 0

while i < 1000:
  imgpath =  path + str(i) + ".jpg"
  imgpath2 =  path2 + str(i) + ".jpg"
  # print(imgpath)
  img = cv2.imread(imgpath, 1)
  img2 = cv2.imread(imgpath2, 1)
  
  rows, columns, channels = img.shape
  rows2, columns2, channels2 = img2.shape

  R = cv2.getRotationMatrix2D((columns/2, rows/2), 45, 1)
  R2 = cv2.getRotationMatrix2D((columns2/2, rows2/2), 45, 1)
  output = cv2.warpAffine(img, R, (columns, rows))
  output2 = cv2.warpAffine(img2, R2, (columns2, rows2))
  cv2.imwrite("/usr/local/src/output/cats/cat" + str(i) + ".jpg", output)
  cv2.imwrite("/usr/local/src/output/dogs/dog" + str(i) + ".jpg", output2)
  i = i + 1




In [0]:
mkdir -p /usr/local/src/output/cats

In [0]:
mkdir -p /usr/local/src/output/dogs

In [0]:
!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import sys

zipname = 'model_rotated'

def zipfolder(foldername, target_dir):            
    zipobj = zipfile.ZipFile(foldername + '.zip', 'w', zipfile.ZIP_DEFLATED)
    rootlen = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zipobj.write(fn, fn[rootlen:])

zipfolder(zipname, '/usr/local/src/pets/models/')

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 2. Create & upload a file text file.
file1 = drive.CreateFile()
file1.SetContentFile(zipname+".zip")
file1.Upload()